In [1]:
import pandas as pd
import os

# === 1. Load comparison list from Excel ===
comparison_df = pd.read_excel('ComparisonList.xlsx', sheet_name='MERP_analysis')

# === 2. Define folder with .merp files ===
data_dir = 'data_analysis'

# === 3. Function to load and preprocess .merp file ===
def load_merp_data(file_name):
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    header_line_idx = None
    for i, line in enumerate(lines):
        if 'MeasurementAreaNo' in line and 'PedNo' in line:
            header_line_idx = i
            break

    if header_line_idx is None:
        raise ValueError(f"Header not found in {file_name}")

    df = pd.read_csv(file_path, sep=';', header=0, skiprows=header_line_idx, low_memory=False)
    df.columns = df.columns.str.strip()
    df_filtered = df[['MeasurementAreaNo', 'tEnter', 'PedNo']].copy()
    df_filtered = df_filtered.astype(str).apply(lambda col: col.str.strip())
    df_filtered = df_filtered[
        df_filtered['MeasurementAreaNo'].str.isnumeric() &
        df_filtered['tEnter'].str.replace('.', '', 1).str.isnumeric() &
        df_filtered['PedNo'].str.isnumeric()
    ]
    df_filtered['MeasurementAreaNo'] = df_filtered['MeasurementAreaNo'].astype(int)
    df_filtered['tEnter'] = df_filtered['tEnter'].astype(float)
    df_filtered['PedNo'] = df_filtered['PedNo'].astype(int)
    return df_filtered

# === 4. Function to build Pedestrian Times and Statistics ===
def create_pedestrian_data(df_filtered):
    t_enter = df_filtered.groupby('PedNo')['tEnter'].min()
    t_leave = df_filtered.groupby('PedNo')['tEnter'].max()
    zone_counts = df_filtered[df_filtered['MeasurementAreaNo'].isin([12, 13])]
    zone_counts = zone_counts.groupby(['PedNo', 'MeasurementAreaNo']).size().unstack(fill_value=0)
    zone_counts.columns = zone_counts.columns.map({12: 'N, times', 13: 'S, times'})
    for col in ['N, times', 'S, times']:
        if col not in zone_counts.columns:
            zone_counts[col] = 0

    ped_time_df = pd.DataFrame({
        'Pedestrian ID': t_enter.index,
        'Entry Time (s)': t_enter.values,
        'Exit Time (s)': t_leave.values,
    })
    ped_time_df['Total Time (s)'] = ped_time_df['Exit Time (s)'] - ped_time_df['Entry Time (s)']
    ped_time_df = ped_time_df.merge(zone_counts, how='left', left_on='Pedestrian ID', right_index=True)
    ped_time_df['N, times'] = ped_time_df['N, times'].fillna(0).astype(int)
    ped_time_df['S, times'] = ped_time_df['S, times'].fillna(0).astype(int)

    stats = {
        'Filename': '',
        'Total Pedestrians': ped_time_df.shape[0],
        'Total Time (s)': ped_time_df['Total Time (s)'].sum(),
        'Average Time (s)': ped_time_df['Total Time (s)'].mean(),
        'Median Time (s)': ped_time_df['Total Time (s)'].median(),
        'N tunnel total': ped_time_df['N, times'].sum(),
        'S tunnel total': ped_time_df['S, times'].sum(),
    }

    return ped_time_df, stats

# === 5. Process each pair in the comparison list ===
for _, row in comparison_df.iterrows():
    base_file = row['Base']
    comp_file = row['Comparison']
    output_filename = f"{row['Filename']}.xlsx"

    print(f"Processing: {base_file} vs {comp_file} -> {output_filename}")

    df_base = load_merp_data(base_file)
    df_comp = load_merp_data(comp_file)

    ped_times_base, stats_base = create_pedestrian_data(df_base)
    ped_times_comp, stats_comp = create_pedestrian_data(df_comp)

    stats_base['Filename'] = base_file
    stats_comp['Filename'] = comp_file

    # === 6. Save results to Excel ===
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        ped_times_base.to_excel(writer, sheet_name='Pedestrian Times - Base', index=False)
        ped_times_comp.to_excel(writer, sheet_name='Pedestrian Times - Comparison', index=False)
        pd.DataFrame([stats_base, stats_comp]).to_excel(writer, sheet_name='Statistics', index=False)

print("✅ All files processed successfully.")


Processing: S13_evening Tram pedestrians.merp vs S14_evening Tram+stops pedestrians.merp -> PedTimeTram_evening.xlsx
Processing: S13_morning Tram pedestrians.merp vs S14_morning Tram+stops pedestrians.merp -> PedTimeTram_morning.xlsx
Processing: S23_evening NewTram pedestrians.merp vs S24_evening NewTram+stops pedestrian.merp -> PedTimeNewTram_evening.xlsx
Processing: S23_morning NewTram pedestrians.merp vs S24_morning NewTram+stops pedestrian.merp -> PedTimeNewTram_morning.xlsx
✅ All files processed successfully.


In [2]:
import pandas as pd
import os

# === 1. Load comparison list ===
comparison_df = pd.read_excel('ComparisonList.xlsx', sheet_name='MERP_analysis')

# === 2. Set directory for .merp files ===
data_dir = 'data_analysis'

# === 3. Function to read and preprocess .merp file ===
def load_merp_data(file_name):
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    header_line_idx = None
    for i, line in enumerate(lines):
        if 'MeasurementAreaNo' in line and 'PedNo' in line:
            header_line_idx = i
            break

    if header_line_idx is None:
        raise ValueError(f"Header not found in {file_name}")

    df = pd.read_csv(file_path, sep=';', header=0, skiprows=header_line_idx, low_memory=False)
    df.columns = df.columns.str.strip()

    # Preprocess relevant columns
    relevant_cols = ['MeasurementAreaNo', 'tEnter', 'PedNo', 'TimeDelay']
    df_filtered = df[relevant_cols].copy()
    df_filtered = df_filtered.astype(str).apply(lambda col: col.str.strip())
    df_filtered = df_filtered[
        df_filtered['MeasurementAreaNo'].str.isnumeric() &
        df_filtered['tEnter'].str.replace('.', '', 1).str.isnumeric() &
        df_filtered['PedNo'].str.isnumeric() &
        df_filtered['TimeDelay'].str.replace('.', '', 1).str.replace('-', '', 1).str.isnumeric()
    ]
    df_filtered['MeasurementAreaNo'] = df_filtered['MeasurementAreaNo'].astype(int)
    df_filtered['tEnter'] = df_filtered['tEnter'].astype(float)
    df_filtered['PedNo'] = df_filtered['PedNo'].astype(int)
    df_filtered['TimeDelay'] = df_filtered['TimeDelay'].astype(float)
    return df_filtered

# === 4. Function to generate pedestrian and statistics data ===
def create_pedestrian_data(df_filtered):
    t_enter = df_filtered.groupby('PedNo')['tEnter'].min()
    t_leave = df_filtered.groupby('PedNo')['tEnter'].max()
    delay_by_ped = df_filtered.groupby('PedNo')['TimeDelay'].sum()

    zone_counts = df_filtered[df_filtered['MeasurementAreaNo'].isin([12, 13])]
    zone_counts = zone_counts.groupby(['PedNo', 'MeasurementAreaNo']).size().unstack(fill_value=0)
    zone_counts.columns = zone_counts.columns.map({12: 'N, times', 13: 'S, times'})
    for col in ['N, times', 'S, times']:
        if col not in zone_counts.columns:
            zone_counts[col] = 0

    ped_time_df = pd.DataFrame({
        'Pedestrian ID': t_enter.index,
        'Entry Time (s)': t_enter.values,
        'Exit Time (s)': t_leave.values,
        'Delay Time (s)': delay_by_ped.reindex(t_enter.index, fill_value=0).values
    })
    ped_time_df['Total Time (s)'] = ped_time_df['Exit Time (s)'] - ped_time_df['Entry Time (s)']
    ped_time_df = ped_time_df.merge(zone_counts, how='left', left_on='Pedestrian ID', right_index=True)
    ped_time_df['N, times'] = ped_time_df['N, times'].fillna(0).astype(int)
    ped_time_df['S, times'] = ped_time_df['S, times'].fillna(0).astype(int)

    stats = {
        'Filename': '',
        'Total Pedestrians': ped_time_df.shape[0],
        'Total Time (s)': ped_time_df['Total Time (s)'].sum(),
        'Average Time (s)': ped_time_df['Total Time (s)'].mean(),
        'Median Time (s)': ped_time_df['Total Time (s)'].median(),
        'N tunnel total': ped_time_df['N, times'].sum(),
        'S tunnel total': ped_time_df['S, times'].sum(),
        'TotalDelayTime': ped_time_df['Delay Time (s)'].sum(),
        'AveDelayTime': ped_time_df['Delay Time (s)'].mean(),
        'MedDelayTime': ped_time_df['Delay Time (s)'].median(),
    }

    return ped_time_df, stats

# === 5. Process each row in the comparison list ===
for _, row in comparison_df.iterrows():
    base_file = row['Base']
    comp_file = row['Comparison']
    output_filename = f"{row['Filename']}.xlsx"

    print(f"Processing: {base_file} vs {comp_file} -> {output_filename}")

    df_base = load_merp_data(base_file)
    df_comp = load_merp_data(comp_file)

    ped_times_base, stats_base = create_pedestrian_data(df_base)
    ped_times_comp, stats_comp = create_pedestrian_data(df_comp)

    stats_base['Filename'] = base_file
    stats_comp['Filename'] = comp_file

    # === 6. Save to Excel ===
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        ped_times_base.to_excel(writer, sheet_name='Pedestrian Times - Base', index=False)
        ped_times_comp.to_excel(writer, sheet_name='Pedestrian Times - Comparison', index=False)
        pd.DataFrame([stats_base, stats_comp]).to_excel(writer, sheet_name='Statistics', index=False)

print("✅ All files processed and saved.")


Processing: S13_evening Tram pedestrians.merp vs S14_evening Tram+stops pedestrians.merp -> PedTimeTram_evening.xlsx
Processing: S13_morning Tram pedestrians.merp vs S14_morning Tram+stops pedestrians.merp -> PedTimeTram_morning.xlsx
Processing: S23_evening NewTram pedestrians.merp vs S24_evening NewTram+stops pedestrian.merp -> PedTimeNewTram_evening.xlsx
Processing: S23_morning NewTram pedestrians.merp vs S24_morning NewTram+stops pedestrian.merp -> PedTimeNewTram_morning.xlsx
✅ All files processed and saved.


In [3]:
import pandas as pd
import os

# === 1. Load comparison list ===
comparison_df = pd.read_excel('ComparisonList.xlsx', sheet_name='MERP_analysis')

# === 2. Set directory for .merp files ===
data_dir = 'data_analysis'

# === 3. Function to read and preprocess .merp file ===
def load_merp_data(file_name):
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    header_line_idx = None
    for i, line in enumerate(lines):
        if 'MeasurementAreaNo' in line and 'PedNo' in line:
            header_line_idx = i
            break

    if header_line_idx is None:
        raise ValueError(f"Header not found in {file_name}")

    df = pd.read_csv(file_path, sep=';', header=0, skiprows=header_line_idx, low_memory=False)
    df.columns = df.columns.str.strip()

    relevant_cols = ['MeasurementAreaNo', 'tEnter', 'PedNo', 'TimeDelay']
    df_filtered = df[relevant_cols].copy()
    df_filtered = df_filtered.astype(str).apply(lambda col: col.str.strip())
    df_filtered = df_filtered[
        df_filtered['MeasurementAreaNo'].str.isnumeric() &
        df_filtered['tEnter'].str.replace('.', '', 1).str.isnumeric() &
        df_filtered['PedNo'].str.isnumeric() &
        df_filtered['TimeDelay'].str.replace('.', '', 1).str.replace('-', '', 1).str.isnumeric()
    ]
    df_filtered['MeasurementAreaNo'] = df_filtered['MeasurementAreaNo'].astype(int)
    df_filtered['tEnter'] = df_filtered['tEnter'].astype(float)
    df_filtered['PedNo'] = df_filtered['PedNo'].astype(int)
    df_filtered['TimeDelay'] = df_filtered['TimeDelay'].astype(float)
    return df_filtered

# === 4. Function to generate pedestrian and statistics data ===
def create_pedestrian_data(df_filtered):
    t_enter = df_filtered.groupby('PedNo')['tEnter'].min()
    t_leave = df_filtered.groupby('PedNo')['tEnter'].max()
    delay_by_ped = df_filtered.groupby('PedNo')['TimeDelay'].sum()

    zone_counts = df_filtered[df_filtered['MeasurementAreaNo'].isin([12, 13])]
    zone_counts = zone_counts.groupby(['PedNo', 'MeasurementAreaNo']).size().unstack(fill_value=0)
    zone_counts.columns = zone_counts.columns.map({12: 'N, times', 13: 'S, times'})
    for col in ['N, times', 'S, times']:
        if col not in zone_counts.columns:
            zone_counts[col] = 0

    ped_time_df = pd.DataFrame({
        'Pedestrian ID': t_enter.index,
        'Entry Time (s)': t_enter.values,
        'Exit Time (s)': t_leave.values,
        'Delay Time (s)': delay_by_ped.reindex(t_enter.index, fill_value=0).values
    })
    ped_time_df['Total Time (s)'] = ped_time_df['Exit Time (s)'] - ped_time_df['Entry Time (s)']
    ped_time_df = ped_time_df.merge(zone_counts, how='left', left_on='Pedestrian ID', right_index=True)
    ped_time_df['N, times'] = ped_time_df['N, times'].fillna(0).astype(int)
    ped_time_df['S, times'] = ped_time_df['S, times'].fillna(0).astype(int)

    stats = {
        'Total Pedestrians': ped_time_df.shape[0],
        'Total Time (s)': ped_time_df['Total Time (s)'].sum(),
        'Average Time (s)': ped_time_df['Total Time (s)'].mean(),
        'Median Time (s)': ped_time_df['Total Time (s)'].median(),
        'N tunnel total': ped_time_df['N, times'].sum(),
        'S tunnel total': ped_time_df['S, times'].sum(),
        'TotalDelayTime': ped_time_df['Delay Time (s)'].sum(),
        'AveDelayTime': ped_time_df['Delay Time (s)'].mean(),
        'MedDelayTime': ped_time_df['Delay Time (s)'].median(),
    }

    return ped_time_df, stats

# === 5. Loop through comparison list ===
all_stats_rows = []

for _, row in comparison_df.iterrows():
    base_file = row['Base']
    comp_file = row['Comparison']
    output_filename = f"{row['Filename']}.xlsx"

    print(f"Processing: {base_file} vs {comp_file} -> {output_filename}")

    df_base = load_merp_data(base_file)
    df_comp = load_merp_data(comp_file)

    ped_times_base, stats_base = create_pedestrian_data(df_base)
    ped_times_comp, stats_comp = create_pedestrian_data(df_comp)

    stats_base['Filename'] = base_file
    stats_base['Output Filename'] = output_filename
    stats_comp['Filename'] = comp_file
    stats_comp['Output Filename'] = output_filename

    all_stats_rows.extend([stats_base, stats_comp])

    # Save individual result
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        ped_times_base.to_excel(writer, sheet_name='Pedestrian Times - Base', index=False)
        ped_times_comp.to_excel(writer, sheet_name='Pedestrian Times - Comparison', index=False)
        pd.DataFrame([stats_base, stats_comp]).to_excel(writer, sheet_name='Statistics', index=False)

# === 6. Save combined Statistics file ===
combined_stats_df = pd.DataFrame(all_stats_rows)
combined_stats_df = combined_stats_df[['Output Filename', 'Filename'] + [col for col in combined_stats_df.columns if col not in ['Output Filename', 'Filename']]]

combined_stats_df.to_excel('PedStatistics.xlsx', index=False)

print("✅ All files processed successfully.")


Processing: S13_evening Tram pedestrians.merp vs S14_evening Tram+stops pedestrians.merp -> PedTimeTram_evening.xlsx
Processing: S13_morning Tram pedestrians.merp vs S14_morning Tram+stops pedestrians.merp -> PedTimeTram_morning.xlsx
Processing: S23_evening NewTram pedestrians.merp vs S24_evening NewTram+stops pedestrian.merp -> PedTimeNewTram_evening.xlsx
Processing: S23_morning NewTram pedestrians.merp vs S24_morning NewTram+stops pedestrian.merp -> PedTimeNewTram_morning.xlsx
✅ All files processed successfully.


In [1]:
import pandas as pd
import os

# === 1. Load comparison list ===
comparison_df = pd.read_excel('ComparisonList.xlsx', sheet_name='MERP_analysis')

# === 2. Set directory for .merp files ===
data_dir = 'data_analysis'

# === 3. Function to read and preprocess .merp file ===
def load_merp_data(file_name):
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    header_line_idx = None
    for i, line in enumerate(lines):
        if 'MeasurementAreaNo' in line and 'PedNo' in line:
            header_line_idx = i
            break

    if header_line_idx is None:
        raise ValueError(f"Header not found in {file_name}")

    df = pd.read_csv(file_path, sep=';', header=0, skiprows=header_line_idx, low_memory=False)
    df.columns = df.columns.str.strip()

    relevant_cols = ['MeasurementAreaNo', 'tEnter', 'PedNo', 'TimeDelay']
    df_filtered = df[relevant_cols].copy()
    df_filtered = df_filtered.astype(str).apply(lambda col: col.str.strip())
    df_filtered = df_filtered[
        df_filtered['MeasurementAreaNo'].str.isnumeric() &
        df_filtered['tEnter'].str.replace('.', '', 1).str.isnumeric() &
        df_filtered['PedNo'].str.isnumeric() &
        df_filtered['TimeDelay'].str.replace('.', '', 1).str.replace('-', '', 1).str.isnumeric()
    ]
    df_filtered['MeasurementAreaNo'] = df_filtered['MeasurementAreaNo'].astype(int)
    df_filtered['tEnter'] = df_filtered['tEnter'].astype(float)
    df_filtered['PedNo'] = df_filtered['PedNo'].astype(int)
    df_filtered['TimeDelay'] = df_filtered['TimeDelay'].astype(float)
    return df_filtered

# === 4. Function to generate pedestrian and statistics data ===
def create_pedestrian_data(df_filtered):
    t_enter = df_filtered.groupby('PedNo')['tEnter'].min()
    t_leave = df_filtered.groupby('PedNo')['tEnter'].max()
    delay_by_ped = df_filtered.groupby('PedNo')['TimeDelay'].sum()

    zone_counts = df_filtered[df_filtered['MeasurementAreaNo'].isin([12, 13])]
    zone_counts = zone_counts.groupby(['PedNo', 'MeasurementAreaNo']).size().unstack(fill_value=0)
    zone_counts.columns = zone_counts.columns.map({12: 'N, times', 13: 'S, times'})
    for col in ['N, times', 'S, times']:
        if col not in zone_counts.columns:
            zone_counts[col] = 0

    ped_time_df = pd.DataFrame({
        'Pedestrian ID': t_enter.index,
        'Entry Time (s)': t_enter.values,
        'Exit Time (s)': t_leave.values,
        'Delay Time (s)': delay_by_ped.reindex(t_enter.index, fill_value=0).values
    })
    ped_time_df['Total Time (s)'] = ped_time_df['Exit Time (s)'] - ped_time_df['Entry Time (s)']
    ped_time_df = ped_time_df.merge(zone_counts, how='left', left_on='Pedestrian ID', right_index=True)
    ped_time_df['N, times'] = ped_time_df['N, times'].fillna(0).astype(int)
    ped_time_df['S, times'] = ped_time_df['S, times'].fillna(0).astype(int)

    stats = {
        'Total Pedestrians': ped_time_df.shape[0],
        'Total Time (s)': ped_time_df['Total Time (s)'].sum(),
        'Average Time (s)': ped_time_df['Total Time (s)'].mean(),
        'Median Time (s)': ped_time_df['Total Time (s)'].median(),
        'N tunnel total': ped_time_df['N, times'].sum(),
        'S tunnel total': ped_time_df['S, times'].sum(),
        'TotalDelayTime': ped_time_df['Delay Time (s)'].sum(),
        'AveDelayTime': ped_time_df['Delay Time (s)'].mean(),
        'MedDelayTime': ped_time_df['Delay Time (s)'].median(),
    }

    return ped_time_df, stats

# === 5. Loop through comparison list ===
all_stats_rows = []

for _, row in comparison_df.iterrows():
    base_file = row['Base']
    comp_file = row['Comparison']
    output_filename = f"{row['Filename']}.xlsx"

    print(f"Processing: {base_file} vs {comp_file} -> {output_filename}")

    df_base = load_merp_data(base_file)
    df_comp = load_merp_data(comp_file)

    ped_times_base, stats_base = create_pedestrian_data(df_base)
    ped_times_comp, stats_comp = create_pedestrian_data(df_comp)

    # Add metadata
    stats_base.update({
        'Output Filename': output_filename,
        'Base': base_file,
        'Comparison': comp_file
    })
    stats_comp.update({
        'Output Filename': output_filename,
        'Base': base_file,
        'Comparison': comp_file
    })

    all_stats_rows.extend([stats_base, stats_comp])

    # Save individual result
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        ped_times_base.to_excel(writer, sheet_name='Pedestrian Times - Base', index=False)
        ped_times_comp.to_excel(writer, sheet_name='Pedestrian Times - Comparison', index=False)
        pd.DataFrame([stats_base, stats_comp]).to_excel(writer, sheet_name='Statistics', index=False)

# === 6. Save combined Statistics file ===
combined_stats_df = pd.DataFrame(all_stats_rows)

# Reorder columns
main_cols = ['Output Filename', 'Base', 'Comparison']
other_cols = [col for col in combined_stats_df.columns if col not in main_cols]
combined_stats_df = combined_stats_df[main_cols + other_cols]

combined_stats_df.to_excel('PedStatistics.xlsx', index=False)

print("✅ All files processed and combined statistics saved to PedStatistics.xlsx")


Processing: S13_evening Tram pedestrians.merp vs S14_evening Tram+stops pedestrians.merp -> MERP_S13_S14_evening_PedTimeTram.xlsx
Processing: S13_morning Tram pedestrians.merp vs S14_morning Tram+stops pedestrians.merp -> MERP_S13_S14_morning_PedTimeTram.xlsx
Processing: S23_evening NewTram pedestrians.merp vs S24_evening NewTram+stops pedestrian.merp -> MERP_S23_S24_evening_PedTimeNewTram.xlsx
Processing: S23_morning NewTram pedestrians.merp vs S24_morning NewTram+stops pedestrian.merp -> MERP_S23_S24_morning_PedTimeNewTram.xlsx
✅ All files processed and combined statistics saved to PedStatistics.xlsx
